# Manager report

1. a) Which movies are longer than 3 hours (180 minutes), show the title and its length?

In [9]:
import duckdb

duckdb_path = "data/sakila.duckdb"

with duckdb.connect(duckdb_path) as conn:
    film_len180 = conn.sql("""
        SELECT
            title,
            length
        FROM
            film
        WHERE
            length > 180;
    """).df()

film_len180.index = film_len180.index + 1
film_len180

,title,length
1,ANALYZE HOOSIERS,181
2,BAKED CLEOPATRA,182
3,CATCH AMISTAD,183
4,CHICAGO NORTH,185
5,CONSPIRACY SPIRIT,184
6,CONTROL ANTHEM,185
7,CRYSTAL BREAKING,184
8,DARN FORRESTER,185
9,FRONTIER CABIN,183
10,GANGS PRIDE,185
